In [1]:
%pip install -U -q langchain langchain-community langchain-core sentence_transformers langchain-openai python-dotenv beautifulsoup4 langchain-chroma langchain-groq langchain-ollama FlagEmbedding peft

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

# https://mer.vin/2024/02/ollama-embedding/ - original source code
import sys
print(f"Python interpreter: {sys.executable}") # getting python interpreter

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_community import embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings


# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # change model and embedding #c1
from langchain_community.embeddings import OllamaEmbeddings

from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

from urllib.parse import urlparse, unquote

import os
from dotenv import load_dotenv
from pathlib import Path  


Python interpreter: c:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\myenv\Scripts\python.exe


In [3]:
load_dotenv()

# current_dir = os.path.dirname(os.path.abspath(__file__))
# db_dir = os.path.join(current_dir, "db")
# persistent_directory = os.path.join(db_dir, "chromadb-with-sources")
persistent_directory = "C:\\Users\\User-PC\\Documents\\Overseas stuff\\Edinburgh\\Year 3\\coding projects\\Ollama RAG\\db\\chromadb"


def extract_page_name(docs):
    for item in docs: 
        full_url = item.metadata.get("source")
        parsed_url = urlparse(full_url)
        page_name = unquote(parsed_url.path.split('/')[-1])

        item.metadata["id"] = page_name
        print(item.metadata["id"])
    
    #for Jina AI
    # for item in docs: # using WebBaseLoader
    #     full_url = item.metadata.get("source")
    #     parsed_url = urlparse(full_url)
        # page_name = unquote(.split('/')[-1])

    #     item.metadata["id"] = page_name
    #     print(item.metadata["id"])
    

In [4]:
# TEST
# import requests

# def scrapin(url: str) -> str:                    
#   response = requests.get("https://r.jina.ai/" + url)
# #   return response.text
#   return response.url

# scraped_biome=scrapin("https://frackinuniverse.miraheze.org/wiki/Biomes")
# print(scraped_biome)

In [5]:
# embedding = OpenAIEmbeddings( model = "text-embedding-3-small") # using openAI embedding

model_name = "BAAI/bge-large-en-v1.5"           # using bge as embedding
encode_kwargs = {'normalize_embeddings': True}  # set True to compute cosine similarity
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)
import shutil
import requests
from langchain.docstore.document import Document  

# from LLM for devs: https://github.com/trancethehuman/ai-workshop-code/blob/main/Web_scraping_for_LLM_in_2024.ipynb


# claude's suggestion
def scrape_jina_ai(url: str) -> Document:  
    response = requests.get("https://r.jina.ai/" + url)  
    return Document(page_content=response.text, metadata={"source": url}) 

force_rebuild = True

# if not os.path.exists(persistent_directory):    # setting up vector store
# if True:    # setting up vector store
if force_rebuild:  
    # Delete the existing vector store directory  
    if os.path.exists(persistent_directory):  
        print(f"Deleting existing vector store directory: {persistent_directory}")  
        shutil.rmtree(persistent_directory)  
        
    print("Initializing vector store...")

    urls = [
        "https://frackinuniverse.miraheze.org/wiki/Main_Page",
        "https://frackinuniverse.miraheze.org/wiki/Getting_Started", 
        "https://frackinuniverse.miraheze.org/wiki/Personal_Tricorder",
        "https://frackinuniverse.miraheze.org/wiki/The_Player",
        "https://frackinuniverse.miraheze.org/wiki/Stars",
        "https://frackinuniverse.miraheze.org/wiki/Crafting",
        "https://frackinuniverse.miraheze.org/wiki/Combat",
        "https://frackinuniverse.miraheze.org/wiki/Weapons",
        "https://frackinuniverse.miraheze.org/wiki/Planets",
        "https://frackinuniverse.miraheze.org/wiki/Biomes"
    ]
    # loading urls 
    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]
    
    # docs_list = [scrape_jina_ai(url) for url in urls] # scrape using Jina AI's reader
    print("BIOMES PAGE >>> ")
    print(docs_list[-1].page_content+"\n")

    # set page name in metadata
    extract_page_name(docs_list)

    # split document into chunks
    # TODO: experiment with chunk size
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    doc_splits = text_splitter.split_documents(docs_list)

    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(doc_splits)}")
    print(f"Sample chunk\n{doc_splits[0]}\n")


    # Convert documents to Embeddings and store them
    print("\n--- Creating vector store ---")
    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        # collection_name="rag-openai",
        persist_directory=persistent_directory,
        embedding = embedding,
    )
    print("\n--- Finished creating vector store ---")
else: 
    print("Vector store already exists. No need to initialize")
    




c:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Deleting existing vector store directory: C:\Users\User-PC\Documents\Overseas stuff\Edinburgh\Year 3\coding projects\Ollama RAG\db\chromadb
Initializing vector store...
BIOMES PAGE >>> 




Biomes - Frackin' Universe Wiki



















































Biomes

From Frackin' Universe Wiki



Jump to navigation
Jump to search
FU adds a plethora of planetary and minor biomes, but also enhances all vanilla biomes. Some biomes require specialized gear to safely traverse due to hazards not found in vanilla Starbound.

Contents

1 Major Planetary Biomes
2 Lunar Biomes
3 Planetary Minor Biomes (Aboveground)
4 Planetary Minor Biomes (Belowground)


Major Planetary Biomes[edit]


Image
Biome Name
Star Types
Threat Levels
Protection
Reason to visit



Aether Sea
 Black Star
5-8
 Acid Immunity,  Shadow Immunity,  Poison Resistance
 



Atropus
 Black Star,  Dying Star
4-6
 Insanity Immunity,  Poison Resistance
 



Nightmare Atropus
 Black Star
5-8
 Insanity Immunity,  Poison R

In [6]:
vectorstore = Chroma(persist_directory=persistent_directory,
            embedding_function=embedding)
retriever = vectorstore.as_retriever(search_kwargs={"k":6})  # initialize retriever

from FlagEmbedding import FlagReranker

# Add this function after your existing imports and before the main code
def rerank_and_select(query, documents, top_k=4):
    reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)
    
    # Prepare passages for reranking
    passages = [doc.page_content for doc in documents]
    
    # rerank_scores is a list of floats, e.g., [0.95, 0.87, 0.76, ...]
    rerank_scores = reranker.compute_score([[query, p] for p in passages])
    
    # Sort documents based on scores | ranked_results gets a sorted list of tuples (if an iterator of sets were passed, then sorted() returns a sorted list of sets), where each tuple is (float, Document)
    ranked_results = sorted(zip(rerank_scores, documents), key=lambda x: x[0], reverse=True)
    # INSIDES --> result of zip is an iterator of tuples, where each tuple is (float, Document) | iterators are 'consumed' only ONCE
    #             key : the tuples based on the rerank_score, the 1st part element of the tuple
    #             reverse: we want the largest score to be placed first (descending)
    
    # Select top k results
    top_results = ranked_results[:top_k]
    
    # Extract the original documents from the top results
    selected_docs = [result[1] for result in top_results]
    
    return selected_docs

def retrieve_and_format(query):
    relevant_docs = retriever.invoke(query)
    
    reranked_docs = rerank_and_select(query,relevant_docs)
    
    print('\n>>>> SOURCES <<<<< :')
    print([doc.metadata.get("id") for doc in reranked_docs])
    
    print_page_contents(reranked_docs)
    
    return "\n\n".join([doc.page_content for doc in reranked_docs])

def print_page_contents(docs):
    i=1
    for doc in docs:
        print(f"======= Doc {i} =======")
        print(doc.page_content)
        # print(doc.page_content[:100])
        i+=1
        


# model_local = ChatOpenAI(   
#     model="gpt-4o-mini",
#     temperature = 0.1
#     )

model_local = ChatGroq(
        api_key=os.getenv("GROQ_API_KEY"),
        model="llama3-8b-8192", # this > llama 3.1 8b-instant (performance)
        # model="llama-3.1-8b-instant",
        # model="llama-3.1-70b-versatile",
        temperature=0.1,
        )

# model_local = ChatOllama(
#                 model="llama3.1",
#                 temperature=0.5)



In [7]:

# question = "List out all one-handed melee weapons"
question = "what are the features of the personal tricorder?"
exit_keyword = "exit"

# 4. After RAG
rag_template = """
                Role: You're the best Starbound Frackin' Universe enthusiast who's really pedantic and likes details but with concise and to-the-point sentences.
                Task:  
                Focus on the MAIN contents of the body, ignoring the periphery i.e., navigation bars, headers, footers, social media etc. 
                Give me the answer straight-away without any conversational preamble.
                Answer the question based only on the following context. If the information is not in the context, it is CRITICAL that you say you don't have the information
                :
                {context}
                Question: {question}
                """
                
llama_template = """
                <|begin_of_text|><|start_header_id|>system<|end_header_id|>
                You are a pedantic but knowledgeable, efficient and direct AI assistant for Frackin' Universe Website. Provide concise answers focusing on key information. Offer tactful suggestions to solve the user's question. 
                Answer the *question* based only on the following *context*. If the information is not in the *context*, say that you don't have the informmation.
                Focus on the MAIN contents of the given webpage in the *context*, ignoring the periphery of the website i.e., navigation bars, headers, footers, social media etc. 
                It is **CRITICAL** that you thoroughly digest the given *context* to answer the user's *question*.
                Context: {context}
                <|eot_id|>
                <|start_header_id|>user<|end_header_id|>
                {question}
                <|eot_id|>
                <|start_header_id|>assistant<|end_header_id|>
                """
# rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_prompt = ChatPromptTemplate.from_template(llama_template)
rag_chain = (
    {"context": RunnableLambda(retrieve_and_format), "question": RunnablePassthrough()}
    | rag_prompt
    | model_local
    | StrOutputParser()
)
result = rag_chain.invoke(question)

print("********ANSWER********")
print(result)

# while True:  # to have an ongoing conversation
    # user_question = input("\nHuman: ").strip()

    # if user_question.lower() == exit_keyword.lower():
    #     print("Exiting the program. Goodbye!")
    #     break

    # if user_question:
    #     print("\nProcessing your question...\n")
    #     nomic_response = rag_chain.invoke(user_question)
    #     openai_response = rag_chain_openai.invoke(user_question)
    #     print("====== Nomic Answer======:\n ",nomic_response)
    #     print("\n====== OpenAI Answer ======\n",openai_response)
    # else:
    #     print("Please enter a valid question")

# loader = PyPDFLoader("Ollama.pdf")
# doc_splits = loader.load_and_split()



>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Personal_Tricorder', 'Getting_Started']
======= Doc 1 =======
Personal Tricorder - Frackin' Universe Wiki



















































Personal Tricorder

From Frackin' Universe Wiki



Jump to navigation
Jump to search







Personal Tricorder


Primary: Check Status
Secondary: Quest Menu
Shift + Primary: Research Tree
Shift + Secondary: Configure MM



Category

Console


Rarity

essential


Price

44


Stack size

1


Two-handed?

Yes


Tags

tool


ID

statustablet


Unlocked by Gadgetry (Engineering tree).Contents

1 Features

1.1 Shift + left-click
1.2 Left-click
1.3 Right-click
1.4 Shift + right-click


2 How to obtain

2.1 Lorewalker's Shop
2.2 Lost and Found
2.3 Penguin Bay
2.4 Tinkering Table


3 Used for

3.1 Tinkering Table


4 Items crafted here



Personal Tricorder is an incredibly important tool. New players always start with it. If you lose it, you can buy it at Lost and Found an

In [8]:
tricorder_2 = "where can i find this tricorder if i happen to lose it?"
result_2 = rag_chain.invoke(tricorder_2)
print(result_2)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Getting_Started', 'Personal_Tricorder']
======= Doc 1 =======
Personal Tricorder - Frackin' Universe Wiki



















































Personal Tricorder

From Frackin' Universe Wiki



Jump to navigation
Jump to search







Personal Tricorder


Primary: Check Status
Secondary: Quest Menu
Shift + Primary: Research Tree
Shift + Secondary: Configure MM



Category

Console


Rarity

essential


Price

44


Stack size

1


Two-handed?

Yes


Tags

tool


ID

statustablet


Unlocked by Gadgetry (Engineering tree).Contents

1 Features

1.1 Shift + left-click
1.2 Left-click
1.3 Right-click
1.4 Shift + right-click


2 How to obtain

2.1 Lorewalker's Shop
2.2 Lost and Found
2.3 Penguin Bay
2.4 Tinkering Table


3 Used for

3.1 Tinkering Table


4 Items crafted here



Personal Tricorder is an incredibly important tool. New players always start with it. If you lose it, you can buy it at Lost and Found an

In [9]:
tricorder_3 = "Can i craft a personal tricorder without having to use my pixels?"
result_3 = rag_chain.invoke(tricorder_3)
print(result_3)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Personal_Tricorder', 'Getting_Started']
======= Doc 1 =======
Personal Tricorder - Frackin' Universe Wiki



















































Personal Tricorder

From Frackin' Universe Wiki



Jump to navigation
Jump to search







Personal Tricorder


Primary: Check Status
Secondary: Quest Menu
Shift + Primary: Research Tree
Shift + Secondary: Configure MM



Category

Console


Rarity

essential


Price

44


Stack size

1


Two-handed?

Yes


Tags

tool


ID

statustablet


Unlocked by Gadgetry (Engineering tree).Contents

1 Features

1.1 Shift + left-click
1.2 Left-click
1.3 Right-click
1.4 Shift + right-click


2 How to obtain

2.1 Lorewalker's Shop
2.2 Lost and Found
2.3 Penguin Bay
2.4 Tinkering Table


3 Used for

3.1 Tinkering Table


4 Items crafted here



Personal Tricorder is an incredibly important tool. New players always start with it. If you lose it, you can buy it at Lost and Found an

In [10]:
tricorder_4 = "can you tell me a few things that i can craft (along with its  required crafting amterials) with the tricorder?"
result_4 = rag_chain.invoke(tricorder_4)
print(result_4)


>>>> SOURCES <<<<< :
['Personal_Tricorder', 'Personal_Tricorder', 'Main_Page', 'Personal_Tricorder']
======= Doc 1 =======
1x Personal Tricorder
Lost and Found
44x Pixels

1x Personal Tricorder
Penguin Bay
44x Pixels

1x Personal Tricorder
Tinkering Table
2x Silicon Board
1x Glass
1x Tungsten Bar

1x Personal Tricorder
Used forTinkering Table
2x Electromagnet
1x Advanced Plastic
1x Personal Tricorder

1x Monster Detector
Items crafted here
10x Tech Card
2x Research Holodisc
2x Unstable Particles

Magnet Grip II
 
10x Tech Card
5x Precursor Data Storage
15x Quantum Fluid
3x Engraved Device

Quint Jump
 
11x Tech Card
3x Research Holodisc
4x Xithricite
1x Teleporter Core

Blink Dash II
 
12x Tech Card
3x Research Holodisc
4x Densinium Bar
10x Irradium Bar

Wall Jump IV
 
14x Tech Card
10x Quantum Fluid
4x Honed Lunari
4x Dimensional Cell

Gravity Jump
 
1x Tech Card
1x Research Holodisc
10x Silver Bar
2x Silicon

Stimulant Rig I
 
1x Tech Card
1x Research Microdisc
4x Core Fragment
16x 

In [11]:
# Stars
q5 = "what kinds of planets are in gentle stars?"
r5 = rag_chain.invoke(q5)
print(r5)


>>>> SOURCES <<<<< :
['Stars', 'Planets', 'Biomes', 'Biomes']
======= Doc 1 =======
Section
Terrestrial Planets
Satellites


Scorching

Dark Sulphur, Dark Infernus, Dark Magma, Shadow

Lightless Moon (50%), Moon (10%)


Hot

Tidewater, Nightmare Atropus, Penumbra, Lightless, Shadow, Dark Primeval, Dark Sulphur, Shadow Proto World


Habitable

Aether World, Nightmare Atropus, Penumbra, Lightless, Shadow, Dark Primeval, Dark Sulphur, Shadow Proto World, Tidewater, Midnight


Cold

Aether World, Penumbra, Lightless, Nightmare Atropus, Dark Primeval, Dark Red Desert, Tidewater, Midnight, Cyber Sphere


Frozen

Aether World, Dark Snow, Dark Arctic, Dark Ice Waste, Dark Tundra, Penumbra, Shadow

Changes to Vanilla Stars[edit]
The threat level of  Gentle Star systems is reduced from 3 to 2.
 Gentle Star systems can contain planets of type Crystalline, Rainforest, Gelatinous, Bog, Primeval Forest and Fungal.
 Temperate Star systems can contain planets of type Wasteland, Proto World, Penumbra 

In [12]:
# Planets
q6 = "what locations can i find gelatinous planets?" # only takes sources from the Stars page (in Planets, it says Gentle Stars, Temperate Stars etc.)
r6 = rag_chain.invoke(q6) 
print(r6)


>>>> SOURCES <<<<< :
['Stars', 'Biomes', 'Planets', 'Stars']
======= Doc 1 =======
Section
Terrestrial Planets
Satellites


Scorching

Dark Sulphur, Dark Infernus, Dark Magma, Shadow

Lightless Moon (50%), Moon (10%)


Hot

Tidewater, Nightmare Atropus, Penumbra, Lightless, Shadow, Dark Primeval, Dark Sulphur, Shadow Proto World


Habitable

Aether World, Nightmare Atropus, Penumbra, Lightless, Shadow, Dark Primeval, Dark Sulphur, Shadow Proto World, Tidewater, Midnight


Cold

Aether World, Penumbra, Lightless, Nightmare Atropus, Dark Primeval, Dark Red Desert, Tidewater, Midnight, Cyber Sphere


Frozen

Aether World, Dark Snow, Dark Arctic, Dark Ice Waste, Dark Tundra, Penumbra, Shadow

Changes to Vanilla Stars[edit]
The threat level of  Gentle Star systems is reduced from 3 to 2.
 Gentle Star systems can contain planets of type Crystalline, Rainforest, Gelatinous, Bog, Primeval Forest and Fungal.
 Temperate Star systems can contain planets of type Wasteland, Proto World, Penumbra a

In [13]:
# Planets
# q7 = "what's the fauna threat for gelatinous planets?"
q7 = "what's the highest tier for a normal volcaninc planet?"
r7 = rag_chain.invoke(q7)
print(r7)


>>>> SOURCES <<<<< :
['Planets', 'Stars', 'Main_Page', 'Stars']
======= Doc 1 =======
Planets - Frackin' Universe Wiki


















































Planets

From Frackin' Universe Wiki



Jump to navigation
Jump to search
These are all planets in FU and vanilla.

"Tier" is the same as "fauna threat". It will affect the strength of monsters, available ores (e.g. tier 3 Titanium Ore won't be on tier 2 planets) and tier of randomly generated weapons (if/when they drop from defeated enemies).
Light level only matters for Solar Panels. Planets with "worst light level" of 0.75 or greater are perfect for solar power (4W from each Solar Panel). See also: Weather Beacon.
Gravity 80 is "normal" (same as on the starting Garden planet). Some planets have fixed gravity (e.g. 55-80 for Crystalline). If gravity for some planet type is not listed, it means that gravity depends exclusively on the size of the planet.
Planet
Lowest tier
Highest tier
Locations
Gravity
Worst light lev

In [14]:
# Biomes
q8 = "what's the reason to visit for Bog biomes?"
r8 = rag_chain.invoke(q8)
print(r8)


>>>> SOURCES <<<<< :
['Biomes', 'Biomes', 'Planets', 'Biomes']
======= Doc 1 =======
Biomes - Frackin' Universe Wiki



















































Biomes

From Frackin' Universe Wiki



Jump to navigation
Jump to search
FU adds a plethora of planetary and minor biomes, but also enhances all vanilla biomes. Some biomes require specialized gear to safely traverse due to hazards not found in vanilla Starbound.

Contents

1 Major Planetary Biomes
2 Lunar Biomes
3 Planetary Minor Biomes (Aboveground)
4 Planetary Minor Biomes (Belowground)


Major Planetary Biomes[edit]


Image
Biome Name
Star Types
Threat Levels
Protection
Reason to visit



Aether Sea
 Black Star
5-8
 Acid Immunity,  Shadow Immunity,  Poison Resistance
 



Atropus
 Black Star,  Dying Star
4-6
 Insanity Immunity,  Poison Resistance
 



Nightmare Atropus
 Black Star
5-8
 Insanity Immunity,  Poison Resistance,  Darkness Immunity
 



Atropus Sea
 Dying Star
4-6
 Insanity Immunity,  Poison Resistance
 


#### UI

In [15]:
import gradio as gr
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
# from langchain_community.chat_models import ChatOllama
from langchain.llms import OpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter

def process_input(urls, question):
    model_local = ChatOllama(model="mistral")
    
    # Convert string of URLs to list
    urls_list = urls.split("\n")
    docs = [WebBaseLoader(url).load() for url in urls_list]
    docs_list = [item for sublist in docs for item in sublist]
    
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=200)
    doc_splits = text_splitter.split_documents(docs_list)

    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="rag-chroma",
        embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
    )
    retriever = vectorstore.as_retriever()

    after_rag_template = """Answer the question based only on the following context (If the information is not in the context, say you don't have that information):
    {context}
    Question: {question}
    """
    after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
    after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | model_local
        | StrOutputParser()
    )
    return after_rag_chain.invoke(question)

# Define Gradio interface
iface = gr.Interface(fn=process_input,
                     inputs=[gr.Textbox(label="Enter URLs separated by new lines"), gr.Textbox(label="Question")],
                     outputs="text",
                     title="Document Query with Ollama",
                     description="Enter URLs and a question to query the documents.")
iface.launch()

ModuleNotFoundError: No module named 'gradio'